In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
import stim
from stimrgs_v1.utils import *
from stimrgs_v1.stabilizer_operations import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from statistics import mean

In [229]:
def count_x_and_z(input_string: str) -> tuple[int, int]:
    """
    Input
        input_string: string to count X and Z ie: 'X___Z'
    Output
        count_x: number of X in string
        count_z: number of Z in string
    """
    # Initialize counters
    count_x = 0
    count_z = 0
    
    # Loop through each character in the string
    for char in input_string:
        # Check for both uppercase and lowercase versions
        if char == 'X' or char == 'x':
            count_x += 1
        elif char == 'Z' or char == 'z':
            count_z += 1
    
    return (count_x, count_z)

def process_stabilizer(num_arms: int, num_bells: int, print_all: bool = False) -> int:
    """
    Input
        num_arms: number of arms
        num_bells: number of bells between row1 and row2, as well as, row3 and row4
        print_all: print process information during simulation
    Output
        Max_bell_found: maximum number of bells founded
    """
    q, circuit = generate_rgs_random(num_nodes=num_arms*4, num_bell_between_row=num_bells)
    Max_bell_found = 0
    
    # Loop over 1st row (i) and 4th row (j)
    for i in range(num_arms):
        for j in range(3*num_arms, 4*num_arms):

            # - - - - - - - - - - - - - Re-initialize after change node to measure - - - - - - - - - - - - - 
            if print_all:
                print(f'Node {i} and {j}')
            # Reset count_bell every time we change node to measure Z
            count_bell = 0

            s = stim.TableauSimulator()
            s.do_circuit(stim.Circuit(circuit))

            # Measure X on all middle nodes
            for middle_node in range(num_arms, num_arms*3):
                s.postselect_x(middle_node, desired_value=False)

            # Measure Z on i and j nodes
            s.postselect_z(i, desired_value=False)
            s.postselect_z(j, desired_value=False)       
            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

            # Check stabilizer after measured, if it has only X and Z, increase count_bell
            # number of stabilizer in s.canonical_stabilizers() = number of nodes
            for stab in s.canonical_stabilizers():
                num_x, num_z = count_x_and_z(str(stab))

                # In cased found Bell (should found 2 times, to counnt it as 1 Bell) ie: 'X__Z', 'Z__X' -> 1 Bell
                if num_x == 1 and num_z == 1:
                    count_bell += 1
                    if print_all:
                        print(f'Stabilizer {stab} has {num_x} X and {num_z} Z')

            if count_bell >= Max_bell_found:
                Max_bell_found = count_bell

            if print_all:
                print(f'Bells: {count_bell//2}')
                print('- - - - - - - - - - - - - - - - - - - - - - - -\n')

    if print_all:
        print(f'Max bells founded: {Max_bell_found//2}')

    return Max_bell_found//2

def expected_process_stabilizer(num_arms: int, num_bells: int, print_all: bool = False) -> int:
    """
    Input
        num_arms: number of arms
        num_bells: number of bells between row1 and row2, as well as, row3 and row4
        print_all: print process information during simulation
    Output
        Max_bell_found: maximum number of bells founded
    """
    q, circuit = generate_rgs_random(num_nodes=num_arms*4, num_bell_between_row=num_bells)
    Max_bell_found = 0
    bell_list = []
    
    # Loop over 1st row (i) and 4th row (j)
    for i in range(num_arms):
        for j in range(3*num_arms, 4*num_arms):

            # - - - - - - - - - - - - - Re-initialize after change node to measure - - - - - - - - - - - - - 
            if print_all:
                print(f'Node {i} and {j}')
            # Reset count_bell every time we change node to measure Z
            count_bell = 0

            s = stim.TableauSimulator()
            s.do_circuit(stim.Circuit(circuit))

            # Measure X on all middle nodes
            for middle_node in range(num_arms, num_arms*3):
                s.postselect_x(middle_node, desired_value=False)

            # Measure Z on i and j nodes
            s.postselect_z(i, desired_value=False)
            s.postselect_z(j, desired_value=False)       
            # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

            # Check stabilizer after measured, if it has only X and Z, increase count_bell
            # number of stabilizer in s.canonical_stabilizers() = number of nodes
            for stab in s.canonical_stabilizers():
                num_x, num_z = count_x_and_z(str(stab))

                # In cased found Bell (should found 2 times, to counnt it as 1 Bell) ie: 'X__Z', 'Z__X' -> 1 Bell
                if num_x == 1 and num_z == 1:
                    count_bell += 1
                    if print_all:
                        print(f'Stabilizer {stab} has {num_x} X and {num_z} Z')

            # collect to calculate the average
            bell_list.append(count_bell)
            if count_bell >= Max_bell_found:
                Max_bell_found = count_bell

            if print_all:
                print(f'Bells: {count_bell//2}')
                print('- - - - - - - - - - - - - - - - - - - - - - - -\n')

    if len(bell_list) != (num_arms**2):
        print(f'len bell_list: {len(bell_list)}')
        print(f'num_arms*2: {num_arms**2}')
        raise ValueError("The number of stabilizer strings is not correct")

    if print_all:
        print(f'Max bells founded: {Max_bell_found//2}')

    return Max_bell_found//2, mean(bell_list)

In [ ]:
# 3 arms 2 bells deterministic
circuit = stim.Circuit('''
    H 0 1 2 3 4 5 6 7 8 9 10 11
    CZ 0 4 0 5 1 3 1 5 2 3 2 4 6 10 6 11 7 9 7 11 8 9 8 10 3 6 4 7 5 8    
''')

q =  [
        'X___ZZ______', # node 0
        '_X_Z_Z______', # node 1
        '__XZZ_______', # node 2
        '_ZZX__Z_____', # node 3
        'Z_Z_X__Z____', # node 4
        'ZZ___X__Z___', # node 5
        '___Z__X___ZZ', # node 6
        '____Z__X_Z_Z', # node 7
        '_____Z__XZZ_', # node 8
        '_______ZZX__', # node 9
        '______Z_Z_X_', # node 10
        '______ZZ___X'  # node 11
    ]

s = stim.TableauSimulator()
s.do_circuit(circuit)

for i in [1,4,7,10]:
    s.postselect_z(i, desired_value=False)

s.postselect_x(3, desired_value=False)
s.postselect_x(6, desired_value=False)
s.postselect_x(5, desired_value=False)
s.postselect_x(8, desired_value=False)

s.canonical_stabilizers()

[stim.PauliString("+X________Z__"),
 stim.PauliString("+Z________X__"),
 stim.PauliString("+_Z__________"),
 stim.PauliString("+__X________Z"),
 stim.PauliString("+__Z________X"),
 stim.PauliString("+___X________"),
 stim.PauliString("+____Z_______"),
 stim.PauliString("+_____X______"),
 stim.PauliString("+______X_____"),
 stim.PauliString("+_______Z____"),
 stim.PauliString("+________X___"),
 stim.PauliString("+__________Z_")]

In [ ]:
# 3 arms 2 bells probabilistic
q, circuit = generate_rgs_random(num_nodes=12, num_bell_between_row=2)
# circuit = '''
#     H 0 1 2 3 4 5 6 7 8 9 10 11
#     CZ 0 4 0 5 1 3 1 5 2 3 2 4 6 10 6 11 7 9 7 11 8 9 8 10 3 6 4 7 5 8    
# '''

# q =  [
#         'X___ZZ______', # node 0
#         '_X_Z_Z______', # node 1
#         '__XZZ_______', # node 2
#         '_ZZX__Z_____', # node 3
#         'Z_Z_X__Z____', # node 4
#         'ZZ___X__Z___', # node 5
#         '___Z__X___ZZ', # node 6
#         '____Z__X_Z_Z', # node 7a
#         '_____Z__XZZ_', # node 8
#         '_______ZZX__', # node 9
#         '______Z_Z_X_', # node 10
#         '______ZZ___X'  # node 11
#     ]

Max_bell_found = 0
for i in [0,1,2]:
    for j in [9, 10, 11]:

        print(f'Node {i} and {j}')
        count_bell = 0

        s = stim.TableauSimulator()
        s.do_circuit(stim.Circuit(circuit))

        for k in range(3, 9):
            s.postselect_x(k, desired_value=False)

        s.postselect_z(i, desired_value=False)
        s.postselect_z(j, desired_value=False)        

        for stab in s.canonical_stabilizers():
            num_x, num_z = count_x_and_z(str(stab))

            if num_x == 1 and num_z == 1:
                count_bell += 1
                print(f'Stabilizer {stab} has {num_x} X and {num_z} Z')

        if count_bell >= Max_bell_found:
            Max_bell_found = count_bell
        print(f'Bells: {count_bell//2}')
        print('- - - - - - - - - - - - - - - - - - - - - - - -\n')

print(f'Max bells founded: {Max_bell_found//2}')

Node 0 and 9
Stabilizer +_X_________Z has 1 X and 1 Z
Stabilizer +__X________Z has 1 X and 1 Z
Bells: 1
- - - - - - - - - - - - - - - - - - - - - - - -

Node 0 and 10
Bells: 0
- - - - - - - - - - - - - - - - - - - - - - - -

Node 0 and 11
Stabilizer +_X_______Z__ has 1 X and 1 Z
Stabilizer +__X______Z__ has 1 X and 1 Z
Bells: 1
- - - - - - - - - - - - - - - - - - - - - - - -

Node 1 and 9
Stabilizer +X__________Z has 1 X and 1 Z
Stabilizer +__X________Z has 1 X and 1 Z
Bells: 1
- - - - - - - - - - - - - - - - - - - - - - - -

Node 1 and 10
Bells: 0
- - - - - - - - - - - - - - - - - - - - - - - -

Node 1 and 11
Stabilizer +X________Z__ has 1 X and 1 Z
Stabilizer +__X______Z__ has 1 X and 1 Z
Bells: 1
- - - - - - - - - - - - - - - - - - - - - - - -

Node 2 and 9
Stabilizer +X__________Z has 1 X and 1 Z
Stabilizer +_X_________Z has 1 X and 1 Z
Bells: 1
- - - - - - - - - - - - - - - - - - - - - - - -

Node 2 and 10
Bells: 0
- - - - - - - - - - - - - - - - - - - - - - - -

Node 2 and 11
Sta

In [225]:
Max_bell_after_experiment = 0
num_arms = 4
num_bells = 2

# process_stabilizer(num_arms=num_arms, num_bells=num_bells, print_all=True)

max_bell, mean_bell = expected_process_stabilizer(num_arms=num_arms, num_bells=num_bells) 
print(f'Max bells founded: {max_bell}')
print(f'Mean bells founded: {mean_bell}')

Max bells founded: 1
Mean bells founded: 1.625


In [230]:
trial = 1000
Max_bell_after_experiment = 0
num_arms = 5
num_bells = 4

# for i in tqdm(range(trial)):
#     Max_bell_found = process_stabilizer(num_arms=num_arms, num_bells=num_bells) 
#     if Max_bell_found > Max_bell_after_experiment:
#         Max_bell_after_experiment = Max_bell_found

# print(f'Final Bell found {Max_bell_after_experiment}')

# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

trial_avg_list = []

for i in tqdm(range(trial)):
    Max_bell_found, Mean_bell_found = expected_process_stabilizer(num_arms=num_arms, num_bells=num_bells) 
    trial_avg_list.append(Mean_bell_found)
    if Max_bell_found > Max_bell_after_experiment:
        Max_bell_after_experiment = Max_bell_found

print(f'Final Bell found {Max_bell_after_experiment}')
print(f'Average Bell found {mean(trial_avg_list)}')

100%|██████████| 1000/1000 [00:11<00:00, 84.91it/s]

Final Bell found 2
Average Bell found 1.48052


In [211]:
# Experiment with number of bells (Fix number of arms)

trial = 1000
num_arms = 6
# result = []

print(f'RGS {num_arms} arms')
# Loop through each number of bells
for bell in range(1, num_arms+1):
    Max_bell_after_experiment = 0

    # Loop through each trial    
    for _ in range(trial):

        Max_bell_found = process_stabilizer(num_arms=num_arms, num_bells=bell) 
        if Max_bell_found > Max_bell_after_experiment:
            Max_bell_after_experiment = Max_bell_found

    # result.append(Max_bell_after_experiment)
    
    print(f'Bells between row {bell}: Max found bells {Max_bell_after_experiment}')

RGS 6 arms
Bells between row 1: Max found bells 5
Bells between row 2: Max found bells 4
Bells between row 3: Max found bells 4
Bells between row 4: Max found bells 4
Bells between row 5: Max found bells 5
Bells between row 6: Max found bells 0


In [206]:
# Experiment with arms (Fix bell between row)

trial = 1000
num_bells = 2

print(f'RGS {num_bells} Bells')
for arm in range(3, 8):
    Max_bell_after_experiment = 0

    for _ in range(trial):
        Max_bell_found = process_stabilizer(num_arms=arm, num_bells=num_bells)
        if Max_bell_found > Max_bell_after_experiment:
            Max_bell_after_experiment = Max_bell_found

    print(f'RGS {arm} arms: Max found bells {Max_bell_after_experiment}')

RGS 2 Bells
RGS 3 arms: Max found bells 2
RGS 4 arms: Max found bells 3
RGS 5 arms: Max found bells 3
RGS 6 arms: Max found bells 4
RGS 7 arms: Max found bells 4


In [210]:
# Experiment with varies both arms and bells

trial = 1000
max_arm = 8

# Loop through each number of bells
for arm in range(3, max_arm+1):
    print(f'RGS {arm} arms')
    for bell in range(2, arm+1):

        Max_bell_after_experiment = 0

        # Loop through each trial    
        for _ in range(trial):

            Max_bell_found = process_stabilizer(num_arms=arm, num_bells=bell) 
            if Max_bell_found > Max_bell_after_experiment:
                Max_bell_after_experiment = Max_bell_found

        
        print(f'{bell} bells between row: Max_bell_found: {Max_bell_after_experiment}')

    print('- - - - - - - - - - - - - - - - - - - - - - -\n')

RGS 3 arms
2 bells between row: Max_bell_found: 2
3 bells between row: Max_bell_found: 0
- - - - - - - - - - - - - - - - - - - - - - -

RGS 4 arms
2 bells between row: Max_bell_found: 3
3 bells between row: Max_bell_found: 3
4 bells between row: Max_bell_found: 0
- - - - - - - - - - - - - - - - - - - - - - -

RGS 5 arms
2 bells between row: Max_bell_found: 4
3 bells between row: Max_bell_found: 4
4 bells between row: Max_bell_found: 2
5 bells between row: Max_bell_found: 0
- - - - - - - - - - - - - - - - - - - - - - -

RGS 6 arms
2 bells between row: Max_bell_found: 4
3 bells between row: Max_bell_found: 4
4 bells between row: Max_bell_found: 3
5 bells between row: Max_bell_found: 5
6 bells between row: Max_bell_found: 0
- - - - - - - - - - - - - - - - - - - - - - -

RGS 7 arms
2 bells between row: Max_bell_found: 4
3 bells between row: Max_bell_found: 3
4 bells between row: Max_bell_found: 4
5 bells between row: Max_bell_found: 3
6 bells between row: Max_bell_found: 4
7 bells between 